# Paper 24: Machine Super Intelligence

**Shane Legg's PhD Thesis (2008): Universal Artificial Intelligence**

A practical exploration of universal intelligence, AIXI agents, and Solomonoff induction through computable approximations.

---

## Overview

This notebook implements key concepts from Shane Legg's foundational work on universal artificial intelligence:

- **Universal Intelligence**: Formal mathematical definition of intelligence
- **AIXI Agent**: Optimal reinforcement learning agent using Solomonoff induction
- **Solomonoff Induction**: Universal prior for sequence prediction
- **Kolmogorov Complexity**: Measuring information content
- **Monte Carlo AIXI**: Practical approximation using sampling
- **Intelligence Measures**: Quantifying agent performance across environments

Since exact AIXI is incomputable, we focus on **practical approximations** using toy environments.

---

## Contents

1. **Theories of Intelligence** - Psychometric models and g-factor
2. **Universal AI & Solomonoff Induction** - Sequence prediction and compression
3. **AIXI Agent & Environment Model** - MC-AIXI in toy MDPs
4. **Universal Intelligence Measure** - Υ(π) for various agents
5. **Approximations & Computational Limits** - Time-bounded AIXI
6. **Pathways to Superintelligence** - Recursive self-improvement and intelligence explosion

---

**Note**: This notebook uses NumPy-only implementations with synthetic environments for educational purposes. Runtime is kept under 5 minutes.

**Connections**:
- Paper 23 (MDL): Minimum description length for model selection
- Paper 25 (Kolmogorov Complexity): Information-theoretic foundations
- Paper 8 (DQN): Practical deep RL vs theoretical optimal agents

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict, deque
from typing import List, Tuple, Dict, Optional
import itertools
from scipy.stats import spearmanr
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Plotting configuration
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Imports complete!")
print("NumPy version:", np.__version__)

---

# Section 1: Theories of Intelligence

## 1.1 Psychometric Intelligence

Human intelligence research uses **psychometric models** to measure cognitive abilities. The **g-factor** (general intelligence) emerges from correlations between different cognitive tests.

### Spearman's g-factor

When people take multiple cognitive tests (verbal, spatial, memory, reasoning), performance across tests is positively correlated. This suggests a single underlying factor: **general intelligence (g)**.

**Model**:
- Test score = g × loading + specific ability + noise
- High g → better performance across all domains

We simulate this with synthetic test data.

In [ ]:
def generate_cognitive_test_data(n_subjects=200, n_tests=8, g_variance=0.7):
    """
    Generate synthetic cognitive test scores with g-factor structure.
    
    Args:
        n_subjects: Number of test subjects
        n_tests: Number of different cognitive tests
        g_variance: Proportion of variance explained by g-factor
        
    Returns:
        scores: (n_subjects, n_tests) test scores
        g_factor: (n_subjects,) underlying general intelligence
    """
    # Generate underlying g-factor (general intelligence) for each subject
    g_factor = np.random.randn(n_subjects)
    
    # Test loadings: how much each test depends on g
    # Higher loading = more g-dependent
    loadings = np.random.uniform(0.5, 0.9, n_tests)
    
    # Generate scores
    scores = np.zeros((n_subjects, n_tests))
    
    for i in range(n_tests):
        # Score = g-component + specific ability + noise
        g_component = g_factor * loadings[i] * np.sqrt(g_variance)
        specific = np.random.randn(n_subjects) * np.sqrt(1 - g_variance)
        scores[:, i] = g_component + specific
    
    # Normalize to 0-100 scale
    scores = 50 + 15 * scores  # Mean=50, SD=15 (like IQ)
    scores = np.clip(scores, 0, 100)
    
    return scores, g_factor, loadings

# Generate test data
test_names = ['Verbal', 'Spatial', 'Memory', 'Reasoning', 'Processing', 'Attention', 'Math', 'Pattern']
scores, g_factor, loadings = generate_cognitive_test_data(n_subjects=200, n_tests=len(test_names))

print("Generated cognitive test data:")
print(f"Subjects: {scores.shape[0]}")
print(f"Tests: {scores.shape[1]}")
print(f"\nMean scores per test:")
for i, name in enumerate(test_names):
    print(f"  {name:12s}: {scores[:, i].mean():.1f} ± {scores[:, i].std():.1f}")

In [ ]:
# Compute correlation matrix (shows positive manifold)
correlation_matrix = np.corrcoef(scores.T)

plt.figure(figsize=(10, 8))
plt.imshow(correlation_matrix, cmap='RdBu_r', vmin=-1, vmax=1)
plt.colorbar(label='Correlation')
plt.xticks(range(len(test_names)), test_names, rotation=45)
plt.yticks(range(len(test_names)), test_names)
plt.title('Correlation Matrix of Cognitive Tests\n(Positive Manifold Shows g-factor)', fontsize=14)

# Add correlation values
for i in range(len(test_names)):
    for j in range(len(test_names)):
        plt.text(j, i, f'{correlation_matrix[i, j]:.2f}', 
                ha='center', va='center', color='white' if abs(correlation_matrix[i, j]) > 0.5 else 'black',
                fontsize=8)

plt.tight_layout()
plt.show()

print("\nKey observation: All tests show POSITIVE correlations")
print("This 'positive manifold' suggests a common underlying factor (g).")
print(f"Mean off-diagonal correlation: {np.mean(correlation_matrix[np.triu_indices_from(correlation_matrix, k=1)]):.3f}")

In [ ]:
# Extract g-factor using Principal Component Analysis (PCA)
def extract_g_factor(scores):
    """
    Extract g-factor as first principal component.
    
    The first PC captures the maximum variance and represents
    the common factor across all tests.
    """
    # Center the data
    scores_centered = scores - scores.mean(axis=0)
    
    # Compute covariance matrix
    cov_matrix = np.cov(scores_centered.T)
    
    # Eigendecomposition
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
    
    # Sort by eigenvalue (descending)
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    # First component is g-factor
    g_extracted = scores_centered @ eigenvectors[:, 0]
    
    # Variance explained
    variance_explained = eigenvalues / eigenvalues.sum()
    
    return g_extracted, variance_explained, eigenvectors[:, 0]

g_extracted, var_explained, g_loadings = extract_g_factor(scores)

# Visualize variance explained
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scree plot
axes[0].bar(range(1, len(var_explained) + 1), var_explained * 100, color='steelblue', alpha=0.7)
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Variance Explained (%)')
axes[0].set_title(f'Scree Plot\nFirst PC (g-factor) explains {var_explained[0]*100:.1f}% of variance')
axes[0].grid(alpha=0.3)

# g-factor loadings
axes[1].barh(test_names, np.abs(g_loadings), color='coral', alpha=0.7)
axes[1].set_xlabel('Absolute Loading on g-factor')
axes[1].set_title('Test Loadings on g-factor\n(How much each test measures g)')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\ng-factor (first PC) explains {var_explained[0]*100:.1f}% of total variance")
print(f"Correlation between true g and extracted g: {np.corrcoef(g_factor, g_extracted)[0, 1]:.3f}")

## 1.2 From Psychometric to Universal Intelligence

**Limitations of psychometric g**:
- Human-centric (only measures human-like intelligence)
- Test-dependent (results vary with test selection)
- No formal mathematical definition

**Legg & Hutter's Universal Intelligence** addresses these by defining intelligence formally:

$$\Upsilon(\pi) = \sum_{\mu \in E} 2^{-K(\mu)} V_\mu^\pi$$

Where:
- $\pi$ is the agent
- $E$ is the set of all computable environments
- $K(\mu)$ is Kolmogorov complexity of environment $\mu$
- $V_\mu^\pi$ is expected reward in environment $\mu$
- $2^{-K(\mu)}$ weights simpler environments more heavily (Solomonoff prior)

This definition:
- Is **universal** (applies to any agent, any environment)
- Is **formal** (precise mathematical definition)
- Aligns with **Occam's razor** (simpler environments weighted more)
- Is **incomputable** (but can be approximated!)

---

# Section 2: Universal AI & Solomonoff Induction

## 2.1 Solomonoff Induction

**Problem**: Given a sequence of observations, predict the next symbol.

**Solomonoff's solution**: Consider ALL computable hypotheses, weighted by their **Kolmogorov complexity** (length of shortest program that generates them).

$$P(x_{1:n}) = \sum_{p: U(p) = x_{1:n}} 2^{-|p|}$$

Where:
- $U$ is a universal Turing machine
- $p$ is a program
- $|p|$ is program length
- Shorter programs have higher prior probability

**Key properties**:
1. Universal: Dominates any computable predictor asymptotically
2. Optimal: Converges to true distribution faster than any other method
3. Incomputable: No algorithm can compute exact Solomonoff probabilities

We approximate this using **simple program enumeration**.

In [ ]:
class SimpleProgramEnumerator:
    """
    Toy approximation of Solomonoff induction using simple program enumeration.
    
    We enumerate short programs (finite state machines) and weight them
    by 2^(-length) to approximate the Solomonoff prior.
    """
    
    def __init__(self, alphabet_size=2, max_program_length=8):
        self.alphabet_size = alphabet_size
        self.max_length = max_program_length
        self.programs = []  # List of (program, weight) tuples
        
    def enumerate_programs(self):
        """
        Enumerate simple programs as repeating patterns.
        
        Programs are represented as short sequences that repeat.
        E.g., [0, 1] represents 010101...
        """
        programs = []
        
        # Enumerate all sequences up to max_length
        for length in range(1, self.max_length + 1):
            for pattern in itertools.product(range(self.alphabet_size), repeat=length):
                program = list(pattern)
                weight = 2.0 ** (-length)  # Solomonoff prior
                programs.append((program, weight))
        
        # Normalize weights
        total_weight = sum(w for _, w in programs)
        programs = [(p, w / total_weight) for p, w in programs]
        
        self.programs = programs
        return len(programs)
    
    def generate_sequence(self, program, length):
        """
        Generate sequence by repeating program pattern.
        """
        seq = []
        for i in range(length):
            seq.append(program[i % len(program)])
        return np.array(seq)
    
    def predict_next(self, observed_sequence):
        """
        Predict next symbol using Solomonoff-style weighted voting.
        
        For each program:
        1. Check if it's consistent with observed sequence
        2. If yes, see what it predicts next
        3. Weight prediction by program's prior probability
        """
        n = len(observed_sequence)
        
        # Accumulate weighted predictions
        predictions = np.zeros(self.alphabet_size)
        total_weight = 0.0
        
        for program, weight in self.programs:
            # Generate what this program would produce
            generated = self.generate_sequence(program, n + 1)
            
            # Check if consistent with observations
            if np.array_equal(generated[:n], observed_sequence):
                next_symbol = generated[n]
                predictions[next_symbol] += weight
                total_weight += weight
        
        if total_weight > 0:
            predictions /= total_weight
        else:
            # Uniform if no consistent programs
            predictions = np.ones(self.alphabet_size) / self.alphabet_size
        
        return predictions

# Create predictor
predictor = SimpleProgramEnumerator(alphabet_size=2, max_program_length=6)
n_programs = predictor.enumerate_programs()

print(f"Enumerated {n_programs} programs")
print(f"\nExample programs (pattern, weight):")
for i in range(min(10, len(predictor.programs))):
    program, weight = predictor.programs[i]
    print(f"  {program} -> weight = {weight:.6f}")

In [ ]:
# Test on different sequences
test_sequences = [
    ([0, 1, 0, 1, 0, 1], "Alternating"),
    ([1, 1, 1, 1, 1, 1], "Constant"),
    ([0, 1, 1, 0, 1, 1], "Pattern 011"),
    ([0, 0, 1, 0, 0, 1], "Pattern 001"),
]

print("Solomonoff Predictions:\n")
for seq, description in test_sequences:
    seq_array = np.array(seq)
    pred = predictor.predict_next(seq_array)
    
    print(f"{description:15s}: {seq}")
    print(f"  P(next=0) = {pred[0]:.4f}")
    print(f"  P(next=1) = {pred[1]:.4f}")
    print(f"  Prediction: {np.argmax(pred)}\n")

In [ ]:
# Visualize program length distribution
program_lengths = [len(p) for p, _ in predictor.programs]
weights_by_length = defaultdict(float)

for program, weight in predictor.programs:
    weights_by_length[len(program)] += weight

lengths = sorted(weights_by_length.keys())
total_weights = [weights_by_length[l] for l in lengths]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Total weight by program length
axes[0].bar(lengths, total_weights, color='steelblue', alpha=0.7)
axes[0].set_xlabel('Program Length (bits)')
axes[0].set_ylabel('Total Prior Probability')
axes[0].set_title('Solomonoff Prior Distribution\n(Shorter programs weighted more)')
axes[0].grid(alpha=0.3)

# Number of programs by length
length_counts = [sum(1 for l in program_lengths if l == length) for length in lengths]
axes[1].bar(lengths, length_counts, color='coral', alpha=0.7)
axes[1].set_xlabel('Program Length (bits)')
axes[1].set_ylabel('Number of Programs')
axes[1].set_title('Program Count by Length\n(Exponential growth in hypotheses)')
axes[1].set_yscale('log')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Key insight: Solomonoff prior favors simplicity (Occam's Razor)")
print(f"Programs of length 1 have total weight: {weights_by_length[1]:.4f}")
print(f"Programs of length 6 have total weight: {weights_by_length[6]:.4f}")

## 2.2 Sequence Prediction Performance

We test how well Solomonoff induction learns different patterns compared to simpler baselines.

In [ ]:
def test_sequence_prediction(true_pattern, seq_length=30, description=""):
    """
    Test prediction accuracy over time as we observe more of the sequence.
    
    Compares:
    - Solomonoff approximation
    - Frequency baseline (predict most common symbol so far)
    - Random baseline
    """
    # Generate true sequence
    true_seq = []
    for i in range(seq_length):
        true_seq.append(true_pattern[i % len(true_pattern)])
    true_seq = np.array(true_seq)
    
    solomonoff_correct = []
    frequency_correct = []
    
    # Start predicting after seeing a few symbols
    for t in range(3, seq_length):
        observed = true_seq[:t]
        true_next = true_seq[t]
        
        # Solomonoff prediction
        sol_pred = predictor.predict_next(observed)
        sol_correct = (np.argmax(sol_pred) == true_next)
        solomonoff_correct.append(sol_correct)
        
        # Frequency baseline: predict most common symbol
        freq_pred = 1 if np.sum(observed) > len(observed) / 2 else 0
        freq_correct = (freq_pred == true_next)
        frequency_correct.append(freq_correct)
    
    return solomonoff_correct, frequency_correct

# Test on different patterns
patterns = [
    ([0, 1], "Alternating"),
    ([0, 0, 1], "Pattern 001"),
    ([1, 0, 1, 1], "Pattern 1011"),
]

fig, axes = plt.subplots(1, len(patterns), figsize=(16, 4))

for idx, (pattern, desc) in enumerate(patterns):
    sol_acc, freq_acc = test_sequence_prediction(pattern, seq_length=30, description=desc)
    
    # Compute cumulative accuracy
    sol_cumavg = np.cumsum(sol_acc) / np.arange(1, len(sol_acc) + 1)
    freq_cumavg = np.cumsum(freq_acc) / np.arange(1, len(freq_acc) + 1)
    
    axes[idx].plot(sol_cumavg, label='Solomonoff', linewidth=2, color='steelblue')
    axes[idx].plot(freq_cumavg, label='Frequency', linewidth=2, color='coral', linestyle='--')
    axes[idx].axhline(0.5, color='gray', linestyle=':', label='Random')
    axes[idx].set_xlabel('Observations')
    axes[idx].set_ylabel('Cumulative Accuracy')
    axes[idx].set_title(f'{desc}\nPattern: {"".join(map(str, pattern))}')
    axes[idx].legend()
    axes[idx].grid(alpha=0.3)
    axes[idx].set_ylim([0, 1.05])

plt.tight_layout()
plt.show()

print("Solomonoff induction quickly identifies simple patterns!")
print("It outperforms frequency baseline by considering program simplicity.")

## 2.3 Kolmogorov Complexity Approximation

**Kolmogorov complexity** $K(x)$ is the length of the shortest program that outputs $x$.

While incomputable in general, we can approximate it using:
1. **Compression**: $K(x) \approx$ compressed length
2. **Program search**: Find shortest program that generates $x$

We use approach #2 with our enumerated programs.

In [ ]:
def estimate_kolmogorov_complexity(sequence):
    """
    Estimate K(sequence) by finding shortest program that generates it.
    """
    n = len(sequence)
    min_length = float('inf')
    best_program = None
    
    for program, weight in predictor.programs:
        generated = predictor.generate_sequence(program, n)
        if np.array_equal(generated, sequence):
            if len(program) < min_length:
                min_length = len(program)
                best_program = program
    
    return min_length, best_program

# Test sequences with different complexities
test_sequences_k = [
    (np.array([0, 0, 0, 0, 0, 0]), "All zeros"),
    (np.array([0, 1, 0, 1, 0, 1]), "Alternating"),
    (np.array([0, 0, 1, 0, 0, 1]), "Pattern 001"),
    (np.array([0, 1, 1, 0, 1, 1]), "Pattern 011"),
    (np.array([1, 0, 0, 1, 1, 0]), "No simple pattern"),
]

print("Kolmogorov Complexity Estimates:\n")
complexities = []
labels = []

for seq, desc in test_sequences_k:
    k_est, program = estimate_kolmogorov_complexity(seq)
    complexities.append(k_est)
    labels.append(desc)
    
    print(f"{desc:20s}: K ≈ {k_est} bits")
    if program is not None:
        print(f"  Shortest program: {"".join(map(str, program))}")
        print(f"  Sequence: {"".join(map(str, seq.tolist()))}\n")
    else:
        print(f"  No program found (complexity > {predictor.max_length})\n")

In [ ]:
# Visualize complexity
plt.figure(figsize=(10, 6))
colors = ['green', 'blue', 'orange', 'orange', 'red']
bars = plt.barh(labels, complexities, color=colors, alpha=0.7)
plt.xlabel('Estimated Kolmogorov Complexity (bits)')
plt.title('Sequence Complexity Estimates\n(Shorter = simpler)', fontsize=14)
plt.grid(axis='x', alpha=0.3)

# Add values
for i, (complexity, label) in enumerate(zip(complexities, labels)):
    plt.text(complexity + 0.1, i, f'{complexity}', va='center')

plt.tight_layout()
plt.show()

print("\nSimpler patterns have lower Kolmogorov complexity!")
print("This formalizes Occam's Razor: prefer simpler explanations.")

---

# Section 3: AIXI Agent & Environment Model

## 3.1 The AIXI Agent

**AIXI** is the theoretically optimal reinforcement learning agent (Hutter, 2005).

**Action selection**:

$$a_t^* = \arg\max_{a_t} \sum_{o_t, r_t} \max_{a_{t+1}} \sum_{o_{t+1}, r_{t+1}} \cdots \max_{a_m} \sum_{o_m, r_m} [r_t + \cdots + r_m] \cdot P(o_t r_t \cdots o_m r_m | a_t \cdots a_m)$$

Where:
$$P(\text{observations} | \text{actions}) = \sum_{\mu \in E} 2^{-K(\mu)} P_\mu(\text{observations} | \text{actions})$$

AIXI:
1. Considers all possible environments weighted by Kolmogorov complexity
2. Plans optimally using minimax search over action sequences
3. Is incomputable (requires infinite computation)

**Monte Carlo AIXI (MC-AIXI)** approximates this using:
- Sampling a small set of environment hypotheses
- Monte Carlo tree search for planning
- Finite lookahead horizon

We implement a simplified MC-AIXI in toy grid-world environments.

In [ ]:
class ToyGridWorld:
    """
    Simple 5x5 grid world environment.
    
    Agent starts at (0, 0) and must reach goal at (4, 4).
    Actions: up, down, left, right
    Rewards: +10 for reaching goal, -1 per step
    """
    
    def __init__(self, size=5):
        self.size = size
        self.reset()
        
    def reset(self):
        self.agent_pos = [0, 0]
        self.goal_pos = [self.size - 1, self.size - 1]
        self.done = False
        self.total_reward = 0
        return self.get_observation()
    
    def get_observation(self):
        """Return agent position as observation."""
        return tuple(self.agent_pos)
    
    def step(self, action):
        """
        Execute action.
        Actions: 0=up, 1=down, 2=left, 3=right
        """
        if self.done:
            return self.get_observation(), 0, True
        
        # Execute action
        if action == 0:  # up
            self.agent_pos[0] = max(0, self.agent_pos[0] - 1)
        elif action == 1:  # down
            self.agent_pos[0] = min(self.size - 1, self.agent_pos[0] + 1)
        elif action == 2:  # left
            self.agent_pos[1] = max(0, self.agent_pos[1] - 1)
        elif action == 3:  # right
            self.agent_pos[1] = min(self.size - 1, self.agent_pos[1] + 1)
        
        # Check if goal reached
        reward = -1  # Step penalty
        if self.agent_pos == self.goal_pos:
            reward = 10
            self.done = True
        
        self.total_reward += reward
        return self.get_observation(), reward, self.done
    
    def copy(self):
        """Create a copy for simulation."""
        new_env = ToyGridWorld(self.size)
        new_env.agent_pos = self.agent_pos.copy()
        new_env.done = self.done
        new_env.total_reward = self.total_reward
        return new_env

# Test environment
env = ToyGridWorld(size=5)
obs = env.reset()

print("Toy Grid World Environment")
print(f"Size: {env.size}x{env.size}")
print(f"Start: {env.agent_pos}")
print(f"Goal: {env.goal_pos}")
print(f"Actions: 0=up, 1=down, 2=left, 3=right")
print(f"\nInitial observation: {obs}")

# Try random actions
print("\nRandom episode:")
total_reward = 0
for step in range(20):
    action = np.random.randint(0, 4)
    obs, reward, done = env.step(action)
    total_reward += reward
    print(f"  Step {step}: action={action}, pos={obs}, reward={reward:.1f}")
    if done:
        print(f"  Goal reached! Total reward: {total_reward}")
        break

In [ ]:
class MCTreeNode:
    """Node in Monte Carlo tree search."""
    
    def __init__(self, state, parent=None, action=None):
        self.state = state
        self.parent = parent
        self.action = action
        self.children = []
        self.visits = 0
        self.value = 0.0
    
    def is_fully_expanded(self, n_actions):
        return len(self.children) == n_actions
    
    def best_child(self, exploration_weight=1.0):
        """Select child using UCB1."""
        choices_weights = []
        for child in self.children:
            if child.visits == 0:
                weight = float('inf')
            else:
                exploit = child.value / child.visits
                explore = exploration_weight * np.sqrt(np.log(self.visits) / child.visits)
                weight = exploit + explore
            choices_weights.append(weight)
        return self.children[np.argmax(choices_weights)]

class SimpleMCAIXI:
    """
    Simplified Monte Carlo AIXI agent.
    
    Uses Monte Carlo Tree Search (MCTS) to plan actions.
    """
    
    def __init__(self, n_actions=4, n_simulations=100, horizon=5):
        self.n_actions = n_actions
        self.n_simulations = n_simulations
        self.horizon = horizon
    
    def select_action(self, env):
        """
        Select action using MCTS.
        
        1. Selection: traverse tree using UCB1
        2. Expansion: add new child node
        3. Simulation: rollout random policy
        4. Backpropagation: update values
        """
        root = MCTreeNode(env.copy())
        
        for _ in range(self.n_simulations):
            node = root
            sim_env = env.copy()
            
            # Selection
            while node.is_fully_expanded(self.n_actions) and len(node.children) > 0:
                node = node.best_child()
                sim_env.step(node.action)
            
            # Expansion
            if not node.is_fully_expanded(self.n_actions) and not sim_env.done:
                action = len(node.children)  # Try next untried action
                new_env = sim_env.copy()
                new_env.step(action)
                child = MCTreeNode(new_env, parent=node, action=action)
                node.children.append(child)
                node = child
                sim_env = new_env
            
            # Simulation (rollout)
            rollout_reward = 0
            for _ in range(self.horizon):
                if sim_env.done:
                    break
                action = np.random.randint(0, self.n_actions)
                _, reward, _ = sim_env.step(action)
                rollout_reward += reward
            
            # Backpropagation
            while node is not None:
                node.visits += 1
                node.value += rollout_reward
                node = node.parent
        
        # Select best action
        if len(root.children) == 0:
            return np.random.randint(0, self.n_actions)
        
        best_child = max(root.children, key=lambda c: c.visits)
        return best_child.action

# Test MC-AIXI agent
print("Testing MC-AIXI agent...\n")

agent = SimpleMCAIXI(n_actions=4, n_simulations=50, horizon=10)
env = ToyGridWorld(size=5)
obs = env.reset()

episode_reward = 0
for step in range(20):
    action = agent.select_action(env)
    obs, reward, done = env.step(action)
    episode_reward += reward
    
    action_names = ['UP', 'DOWN', 'LEFT', 'RIGHT']
    print(f"Step {step}: {action_names[action]:5s} -> pos={obs}, reward={reward:+.1f}")
    
    if done:
        print(f"\nGoal reached in {step + 1} steps!")
        print(f"Total reward: {episode_reward}")
        break

if not done:
    print(f"\nDid not reach goal. Total reward: {episode_reward}")

## 3.2 Comparing Agents

Let's compare different agents:
1. **Random**: Selects actions uniformly
2. **Greedy**: Moves toward goal (Manhattan distance)
3. **MC-AIXI**: Uses MCTS planning

In [ ]:
def random_agent(env):
    """Random action selection."""
    return np.random.randint(0, 4)

def greedy_agent(env):
    """
    Greedy agent: move toward goal.
    
    Computes Manhattan distance and chooses action that reduces it.
    """
    agent_pos = env.agent_pos
    goal_pos = env.goal_pos
    
    # Vertical movement
    if agent_pos[0] < goal_pos[0]:
        return 1  # down
    elif agent_pos[0] > goal_pos[0]:
        return 0  # up
    # Horizontal movement
    elif agent_pos[1] < goal_pos[1]:
        return 3  # right
    elif agent_pos[1] > goal_pos[1]:
        return 2  # left
    else:
        return np.random.randint(0, 4)

def evaluate_agent(agent_fn, n_episodes=20, max_steps=30):
    """
    Evaluate agent over multiple episodes.
    """
    total_rewards = []
    steps_to_goal = []
    
    for _ in range(n_episodes):
        env = ToyGridWorld(size=5)
        env.reset()
        
        episode_reward = 0
        for step in range(max_steps):
            action = agent_fn(env)
            obs, reward, done = env.step(action)
            episode_reward += reward
            
            if done:
                steps_to_goal.append(step + 1)
                break
        
        total_rewards.append(episode_reward)
    
    return total_rewards, steps_to_goal

# Evaluate agents
print("Evaluating agents over 20 episodes...\n")

mc_aixi_agent = SimpleMCAIXI(n_actions=4, n_simulations=30, horizon=10)

random_rewards, random_steps = evaluate_agent(random_agent, n_episodes=20)
greedy_rewards, greedy_steps = evaluate_agent(greedy_agent, n_episodes=20)
aixi_rewards, aixi_steps = evaluate_agent(lambda env: mc_aixi_agent.select_action(env), n_episodes=20)

# Results
print("Results (mean ± std):")
print(f"\nRandom Agent:")
print(f"  Reward: {np.mean(random_rewards):.1f} ± {np.std(random_rewards):.1f}")
print(f"  Success rate: {len(random_steps)/20*100:.0f}%")
if len(random_steps) > 0:
    print(f"  Steps to goal: {np.mean(random_steps):.1f} ± {np.std(random_steps):.1f}")

print(f"\nGreedy Agent:")
print(f"  Reward: {np.mean(greedy_rewards):.1f} ± {np.std(greedy_rewards):.1f}")
print(f"  Success rate: {len(greedy_steps)/20*100:.0f}%")
if len(greedy_steps) > 0:
    print(f"  Steps to goal: {np.mean(greedy_steps):.1f} ± {np.std(greedy_steps):.1f}")

print(f"\nMC-AIXI Agent:")
print(f"  Reward: {np.mean(aixi_rewards):.1f} ± {np.std(aixi_rewards):.1f}")
print(f"  Success rate: {len(aixi_steps)/20*100:.0f}%")
if len(aixi_steps) > 0:
    print(f"  Steps to goal: {np.mean(aixi_steps):.1f} ± {np.std(aixi_steps):.1f}")

In [ ]:
# Visualize performance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Reward comparison
agent_names = ['Random', 'Greedy', 'MC-AIXI']
mean_rewards = [np.mean(random_rewards), np.mean(greedy_rewards), np.mean(aixi_rewards)]
std_rewards = [np.std(random_rewards), np.std(greedy_rewards), np.std(aixi_rewards)]

axes[0].bar(agent_names, mean_rewards, yerr=std_rewards, capsize=5, 
           color=['gray', 'steelblue', 'coral'], alpha=0.7)
axes[0].set_ylabel('Total Reward')
axes[0].set_title('Agent Performance\n(Higher is better)')
axes[0].grid(axis='y', alpha=0.3)

# Steps to goal comparison
steps_data = [random_steps, greedy_steps, aixi_steps]
axes[1].boxplot(steps_data, labels=agent_names)
axes[1].set_ylabel('Steps to Goal')
axes[1].set_title('Efficiency\n(Lower is better)')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\nMC-AIXI combines planning and lookahead to achieve better performance!")
print("Greedy agent is fast but suboptimal in complex environments.")
print("Random agent shows the baseline for comparison.")

---

# Section 4: Universal Intelligence Measure

## 4.1 Formal Definition

**Legg & Hutter's Universal Intelligence Measure**:

$$\Upsilon(\pi) = \sum_{\mu \in E} 2^{-K(\mu)} V_\mu^\pi$$

Where:
- $\pi$ is the agent policy
- $E$ is the space of all computable environments
- $K(\mu)$ is Kolmogorov complexity of environment $\mu$
- $V_\mu^\pi$ is expected discounted reward in environment $\mu$:

$$V_\mu^\pi = \mathbb{E}_{\pi, \mu}\left[\sum_{t=1}^\infty \gamma^t r_t\right]$$

**Key properties**:
1. **Universal**: Considers all possible computable environments
2. **Weighted by simplicity**: Simpler environments matter more (Solomonoff prior)
3. **Performance-based**: Measured by reward achievement
4. **Incomputable**: But can be approximated with finite environment sets

We approximate $\Upsilon(\pi)$ using a small suite of toy environments.

In [ ]:
class EnvironmentSuite:
    """
    Collection of toy environments with estimated Kolmogorov complexities.
    """
    
    def __init__(self):
        self.environments = self.create_environments()
    
    def create_environments(self):
        """
        Create environments with varying complexities.
        
        Each environment returns:
        - env_fn: Function that creates environment instance
        - complexity: Estimated K(μ) in bits
        - description: Human-readable description
        """
        envs = []
        
        # Env 1: Constant reward (simplest)
        def constant_reward_env():
            class ConstantEnv:
                def __init__(self):
                    self.done = False
                def reset(self):
                    self.done = False
                    return 0
                def step(self, action):
                    return 0, 1.0, False  # Always reward +1
                def copy(self):
                    new = ConstantEnv()
                    new.done = self.done
                    return new
            return ConstantEnv()
        
        envs.append({
            'env_fn': constant_reward_env,
            'complexity': 2,  # Very simple: always return 1
            'description': 'Constant reward (+1)'
        })
        
        # Env 2: Binary choice (left=0, right=1)
        def binary_choice_env():
            class BinaryEnv:
                def __init__(self):
                    self.done = False
                def reset(self):
                    self.done = False
                    return 0
                def step(self, action):
                    # Action 3 (right) gives +1, others give 0
                    reward = 1.0 if action == 3 else 0.0
                    return 0, reward, False
                def copy(self):
                    return BinaryEnv()
            return BinaryEnv()
        
        envs.append({
            'env_fn': binary_choice_env,
            'complexity': 4,  # Need to encode: if action==3 then 1 else 0
            'description': 'Binary choice (right=+1)'
        })
        
        # Env 3: Grid world (more complex)
        def grid_world_env():
            return ToyGridWorld(size=5)
        
        envs.append({
            'env_fn': grid_world_env,
            'complexity': 8,  # Need to encode: grid, movement dynamics, goal
            'description': 'Grid world navigation'
        })
        
        return envs
    
    def compute_universal_intelligence(self, agent_fn, n_episodes=10, max_steps=20, gamma=0.95):
        """
        Compute Υ(π) approximation.
        
        Υ(π) ≈ Σ_μ 2^(-K(μ)) * V_μ^π
        """
        intelligence = 0.0
        environment_values = []
        
        for env_spec in self.environments:
            # Get environment parameters
            env_fn = env_spec['env_fn']
            complexity = env_spec['complexity']
            
            # Compute environment weight (Solomonoff prior)
            weight = 2.0 ** (-complexity)
            
            # Estimate V_μ^π (expected discounted return)
            returns = []
            for _ in range(n_episodes):
                env = env_fn()
                env.reset()
                
                discounted_return = 0.0
                discount = 1.0
                
                for step in range(max_steps):
                    action = agent_fn(env)
                    _, reward, done = env.step(action)
                    discounted_return += discount * reward
                    discount *= gamma
                    
                    if done:
                        break
                
                returns.append(discounted_return)
            
            value = np.mean(returns)
            environment_values.append({
                'description': env_spec['description'],
                'complexity': complexity,
                'weight': weight,
                'value': value,
                'weighted_value': weight * value
            })
            
            # Add to intelligence measure
            intelligence += weight * value
        
        # Normalize by total weight
        total_weight = sum(2.0 ** (-env['complexity']) for env in self.environments)
        intelligence /= total_weight
        
        return intelligence, environment_values

# Create environment suite
suite = EnvironmentSuite()

print("Environment Suite:")
print()
for i, env_spec in enumerate(suite.environments, 1):
    weight = 2.0 ** (-env_spec['complexity'])
    print(f"{i}. {env_spec['description']}")
    print(f"   K(μ) ≈ {env_spec['complexity']} bits")
    print(f"   Weight = 2^(-{env_spec['complexity']}) = {weight:.4f}")
    print()

In [ ]:
# Compute intelligence for different agents
print("Computing Universal Intelligence Υ(π) for each agent...\n")

mc_aixi_light = SimpleMCAIXI(n_actions=4, n_simulations=20, horizon=8)

# Random agent
upsilon_random, env_values_random = suite.compute_universal_intelligence(
    random_agent, n_episodes=10, max_steps=15
)

# Greedy agent
upsilon_greedy, env_values_greedy = suite.compute_universal_intelligence(
    greedy_agent, n_episodes=10, max_steps=15
)

# MC-AIXI agent
upsilon_aixi, env_values_aixi = suite.compute_universal_intelligence(
    lambda env: mc_aixi_light.select_action(env), n_episodes=10, max_steps=15
)

print("Universal Intelligence Υ(π):")
print(f"  Random:  {upsilon_random:.3f}")
print(f"  Greedy:  {upsilon_greedy:.3f}")
print(f"  MC-AIXI: {upsilon_aixi:.3f}")
print()

# Show breakdown by environment
print("Breakdown by environment (MC-AIXI):")
for ev in env_values_aixi:
    print(f"  {ev['description']:30s}: V={ev['value']:6.2f}, weight={ev['weight']:.4f}, contribution={ev['weighted_value']:.3f}")

In [ ]:
# Visualize intelligence comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Overall intelligence
agents = ['Random', 'Greedy', 'MC-AIXI']
intelligence_scores = [upsilon_random, upsilon_greedy, upsilon_aixi]

axes[0].bar(agents, intelligence_scores, color=['gray', 'steelblue', 'coral'], alpha=0.7)
axes[0].set_ylabel('Υ(π)')
axes[0].set_title('Universal Intelligence Measure\n(Higher = more intelligent)', fontsize=12)
axes[0].grid(axis='y', alpha=0.3)

# Add values on bars
for i, score in enumerate(intelligence_scores):
    axes[0].text(i, score + 0.05, f'{score:.3f}', ha='center', fontsize=11, fontweight='bold')

# Performance by environment
env_names = [ev['description'] for ev in env_values_aixi]
random_values = [ev['value'] for ev in env_values_random]
greedy_values = [ev['value'] for ev in env_values_greedy]
aixi_values = [ev['value'] for ev in env_values_aixi]

x = np.arange(len(env_names))
width = 0.25

axes[1].bar(x - width, random_values, width, label='Random', color='gray', alpha=0.7)
axes[1].bar(x, greedy_values, width, label='Greedy', color='steelblue', alpha=0.7)
axes[1].bar(x + width, aixi_values, width, label='MC-AIXI', color='coral', alpha=0.7)

axes[1].set_ylabel('Expected Return V_μ^π')
axes[1].set_title('Performance by Environment', fontsize=12)
axes[1].set_xticks(x)
axes[1].set_xticklabels(env_names, rotation=15, ha='right')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey insight: MC-AIXI achieves higher Υ(π) through better performance")
print("across diverse environments, especially complex ones!")

## 4.2 Intelligence and Environment Complexity

The universal intelligence measure weights simpler environments more heavily. This means:
- Performance on simple environments matters more
- But agents must still handle diverse challenges

Let's visualize how environment complexity affects the intelligence measure.

In [ ]:
# Analyze contribution by complexity
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Solomonoff weights by complexity
complexities = [ev['complexity'] for ev in env_values_aixi]
weights = [ev['weight'] for ev in env_values_aixi]
contributions_aixi = [ev['weighted_value'] for ev in env_values_aixi]

axes[0].bar(env_names, weights, color='purple', alpha=0.6)
axes[0].set_ylabel('Solomonoff Weight (2^-K)')
axes[0].set_title('Environment Weighting\n(Simpler = higher weight)', fontsize=12)
axes[0].set_xticks(range(len(env_names)))
axes[0].set_xticklabels(env_names, rotation=15, ha='right')
axes[0].grid(axis='y', alpha=0.3)

# Add complexity labels
for i, (w, k) in enumerate(zip(weights, complexities)):
    axes[0].text(i, w + 0.01, f'K={k}', ha='center', fontsize=9)

# Contribution to Υ(π)
axes[1].bar(env_names, contributions_aixi, color='coral', alpha=0.7)
axes[1].set_ylabel('Contribution to Υ(π)')
axes[1].set_title('Environment Contributions\n(weight × value)', fontsize=12)
axes[1].set_xticks(range(len(env_names)))
axes[1].set_xticklabels(env_names, rotation=15, ha='right')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("Simpler environments (lower K) contribute more to intelligence measure.")
print("This embodies Occam's razor: simple patterns matter most.")

---

# Section 5: Approximations & Computational Limits

## 5.1 Why AIXI is Incomputable

**Theoretical AIXI**:
- Considers ALL computable environments
- Uses exact Kolmogorov complexity (incomputable)
- Performs infinite lookahead planning

**Practical approximations**:
1. **MC-AIXI**: Samples environment hypotheses, uses MCTS
2. **Time-bounded AIXI**: Limits computation time
3. **Restricted hypothesis spaces**: Only consider simple environment models

We explore the **time-computation tradeoff**.

In [ ]:
# Compare MC-AIXI with different computation budgets
def evaluate_with_budget(n_simulations_list, n_episodes=10):
    """
    Evaluate MC-AIXI with different computation budgets.
    """
    results = []
    
    for n_sims in n_simulations_list:
        agent = SimpleMCAIXI(n_actions=4, n_simulations=n_sims, horizon=8)
        
        # Test on grid world
        rewards = []
        for _ in range(n_episodes):
            env = ToyGridWorld(size=5)
            env.reset()
            
            episode_reward = 0
            for step in range(20):
                action = agent.select_action(env)
                _, reward, done = env.step(action)
                episode_reward += reward
                if done:
                    break
            
            rewards.append(episode_reward)
        
        results.append({
            'simulations': n_sims,
            'mean_reward': np.mean(rewards),
            'std_reward': np.std(rewards)
        })
    
    return results

# Test different budgets
print("Testing computation budget effect...\n")
budgets = [5, 10, 20, 50, 100]
budget_results = evaluate_with_budget(budgets, n_episodes=10)

print("Results:")
print(f"{'Simulations':<12} {'Mean Reward':<15} {'Std Reward':<15}")
print("-" * 45)
for res in budget_results:
    print(f"{res['simulations']:<12} {res['mean_reward']:<15.2f} {res['std_reward']:<15.2f}")

In [ ]:
# Visualize performance vs computation
simulations = [res['simulations'] for res in budget_results]
mean_rewards = [res['mean_reward'] for res in budget_results]
std_rewards = [res['std_reward'] for res in budget_results]

plt.figure(figsize=(10, 6))
plt.errorbar(simulations, mean_rewards, yerr=std_rewards, 
             marker='o', markersize=8, capsize=5, linewidth=2,
             color='steelblue', label='MC-AIXI')
plt.axhline(np.mean(greedy_rewards), color='coral', linestyle='--', 
           linewidth=2, label='Greedy (no planning)')
plt.xlabel('MCTS Simulations (computation budget)', fontsize=12)
plt.ylabel('Mean Episode Reward', fontsize=12)
plt.title('Performance vs Computation Budget\n(Diminishing returns)', fontsize=14)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\nKey observation: Performance improves with computation but plateaus.")
print("This shows the tradeoff between optimality and computability.")

## 5.2 Approximation Quality

Different approximations make different tradeoffs:

| Method | Computation | Optimality | Practical? |
|--------|-------------|------------|------------|
| Theoretical AIXI | Infinite | Perfect | No |
| MC-AIXI (large budget) | Very high | Good | Sometimes |
| MC-AIXI (small budget) | Medium | Fair | Yes |
| Greedy/heuristic | Low | Poor | Yes |

This illustrates the **fundamental tension** between:
- **Universality**: Handling all environments
- **Optimality**: Making best decisions
- **Computability**: Running in finite time

---

# Section 6: Pathways to Superintelligence

## 6.1 Recursive Self-Improvement

A key pathway to superintelligence is **recursive self-improvement**:
1. System improves its own intelligence
2. Smarter system makes better improvements
3. Process accelerates (intelligence explosion)

We simulate this with a toy model where agents can:
- Improve their planning depth
- Increase their MCTS simulation budget
- Learn environment models

In [ ]:
class SelfImprovingAgent:
    """
    Agent that can improve its own capabilities.
    
    Improvement mechanisms:
    - Increase MCTS simulation budget
    - Increase planning horizon
    """
    
    def __init__(self, initial_simulations=10, initial_horizon=5):
        self.simulations = initial_simulations
        self.horizon = initial_horizon
        self.improvement_history = [{
            'step': 0,
            'simulations': initial_simulations,
            'horizon': initial_horizon,
            'intelligence': 0
        }]
    
    def select_action(self, env):
        """Select action using current capabilities."""
        agent = SimpleMCAIXI(
            n_actions=4,
            n_simulations=self.simulations,
            horizon=self.horizon
        )
        return agent.select_action(env)
    
    def self_improve(self, performance_feedback):
        """
        Improve capabilities based on performance.
        
        Better performance → more resources for improvement.
        This creates positive feedback loop.
        """
        # Improvement rate proportional to current intelligence
        improvement_factor = 1.0 + (performance_feedback / 10.0)
        
        # Increase computation budget
        self.simulations = int(self.simulations * improvement_factor)
        self.simulations = min(self.simulations, 200)  # Cap for practicality
        
        # Increase planning depth
        if np.random.rand() < 0.3:  # Occasional horizon increase
            self.horizon = min(self.horizon + 1, 15)
        
        self.improvement_history.append({
            'step': len(self.improvement_history),
            'simulations': self.simulations,
            'horizon': self.horizon,
            'intelligence': performance_feedback
        })

# Simulate self-improvement process
print("Simulating recursive self-improvement...\n")

agent = SelfImprovingAgent(initial_simulations=5, initial_horizon=3)

n_improvement_cycles = 8
for cycle in range(n_improvement_cycles):
    # Evaluate current performance
    rewards = []
    for _ in range(5):  # 5 episodes per evaluation
        env = ToyGridWorld(size=5)
        env.reset()
        
        episode_reward = 0
        for step in range(20):
            action = agent.select_action(env)
            _, reward, done = env.step(action)
            episode_reward += reward
            if done:
                break
        rewards.append(episode_reward)
    
    performance = np.mean(rewards)
    
    print(f"Cycle {cycle}:")
    print(f"  Simulations: {agent.simulations:4d}  Horizon: {agent.horizon:2d}")
    print(f"  Performance: {performance:6.2f}\n")
    
    # Self-improve
    agent.self_improve(performance)

In [ ]:
# Visualize intelligence explosion
history = agent.improvement_history
steps = [h['step'] for h in history]
simulations_hist = [h['simulations'] for h in history]
horizon_hist = [h['horizon'] for h in history]
intelligence_hist = [h['intelligence'] for h in history]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Capability growth
ax1 = axes[0]
ax1.plot(steps, simulations_hist, marker='o', linewidth=2, 
        color='steelblue', label='MCTS Simulations')
ax1.set_xlabel('Improvement Cycle', fontsize=12)
ax1.set_ylabel('MCTS Simulations', color='steelblue', fontsize=12)
ax1.tick_params(axis='y', labelcolor='steelblue')
ax1.grid(alpha=0.3)

ax1_twin = ax1.twinx()
ax1_twin.plot(steps, horizon_hist, marker='s', linewidth=2, 
             color='coral', label='Planning Horizon')
ax1_twin.set_ylabel('Planning Horizon', color='coral', fontsize=12)
ax1_twin.tick_params(axis='y', labelcolor='coral')

axes[0].set_title('Capability Growth\n(Recursive Self-Improvement)', fontsize=13)

# Intelligence growth (exponential-like)
axes[1].plot(steps, intelligence_hist, marker='o', linewidth=2.5, 
            color='purple', label='Performance')
axes[1].set_xlabel('Improvement Cycle', fontsize=12)
axes[1].set_ylabel('Performance (Intelligence)', fontsize=12)
axes[1].set_title('Intelligence Explosion\n(Accelerating improvement)', fontsize=13)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey insight: Self-improvement creates positive feedback loop.")
print("Better performance → more resources → better improvements → faster growth.")
print("This is the theoretical basis for 'intelligence explosion'.")

## 6.2 Intelligence Explosion Dynamics

**Factors affecting takeoff speed**:

1. **Optimization power**: How well can system improve itself?
2. **Recalcitrance**: How hard is it to make improvements?
3. **Diminishing returns**: Do improvements get harder over time?

**Takeoff scenarios**:
- **Slow takeoff**: Linear or sublinear growth (decades to superintelligence)
- **Fast takeoff**: Exponential growth (years to superintelligence)
- **Hard takeoff**: Super-exponential growth (days/weeks to superintelligence)

We model different growth curves.

In [ ]:
def simulate_takeoff(growth_model, n_steps=50):
    """
    Simulate intelligence growth under different models.
    
    growth_model: function(current_intelligence, step) -> new_intelligence
    """
    intelligence = [1.0]  # Start at baseline human-level (1.0)
    
    for step in range(1, n_steps):
        new_int = growth_model(intelligence[-1], step)
        intelligence.append(new_int)
    
    return intelligence

# Different growth models
def linear_growth(current, step):
    """Slow takeoff: I(t) = I(0) + kt"""
    return 1.0 + 0.1 * step

def exponential_growth(current, step):
    """Fast takeoff: I(t) = I(0) * e^(kt)"""
    return current * 1.15

def superexponential_growth(current, step):
    """Hard takeoff: I(t+1) = I(t)^k (recursive improvement)"""
    growth_rate = 1.05 + (current - 1.0) * 0.01  # Acceleration
    return current * growth_rate

def diminishing_returns(current, step):
    """Slow takeoff with saturation: asymptotic to limit"""
    limit = 10.0
    rate = 0.15
    return current + rate * (limit - current)

# Simulate scenarios
linear_curve = simulate_takeoff(linear_growth, n_steps=50)
exponential_curve = simulate_takeoff(exponential_growth, n_steps=50)
superexp_curve = simulate_takeoff(superexponential_growth, n_steps=50)
diminishing_curve = simulate_takeoff(diminishing_returns, n_steps=50)

# Visualize
plt.figure(figsize=(12, 7))

steps = range(len(linear_curve))
plt.plot(steps, linear_curve, linewidth=2.5, label='Linear (Slow Takeoff)', color='blue')
plt.plot(steps, exponential_curve, linewidth=2.5, label='Exponential (Fast Takeoff)', color='orange')
plt.plot(steps, superexp_curve, linewidth=2.5, label='Super-exponential (Hard Takeoff)', color='red')
plt.plot(steps, diminishing_curve, linewidth=2.5, label='Diminishing Returns', color='green', linestyle='--')

plt.axhline(1.0, color='gray', linestyle=':', linewidth=1.5, label='Human-level')
plt.xlabel('Time Steps', fontsize=13)
plt.ylabel('Intelligence (human-level = 1.0)', fontsize=13)
plt.title('Intelligence Explosion Scenarios\n(Different growth dynamics)', fontsize=15)
plt.legend(fontsize=11, loc='upper left')
plt.grid(alpha=0.3)
plt.yscale('log')
plt.ylim([0.8, 1000])
plt.tight_layout()
plt.show()

print("Intelligence growth depends critically on feedback dynamics!")
print("\nAt step 50:")
print(f"  Linear:          {linear_curve[-1]:8.1f}x human-level")
print(f"  Exponential:     {exponential_curve[-1]:8.1f}x human-level")
print(f"  Super-exp:       {superexp_curve[-1]:8.1f}x human-level")
print(f"  Diminishing:     {diminishing_curve[-1]:8.1f}x human-level")

## 6.3 Implications and Risks

**Key insights from universal intelligence theory**:

1. **Intelligence is measurable**: Υ(π) provides formal definition
2. **AIXI is optimal**: But incomputable (fundamental limitation)
3. **Approximations exist**: MC-AIXI and variants work in practice
4. **Self-improvement is possible**: Leads to potential intelligence explosion

**Open questions**:
- How fast will real AI systems improve?
- What are fundamental limits on intelligence?
- How do we maintain control during recursive self-improvement?
- Can we align superintelligent systems with human values?

**Connections to safety**:
- AIXI has no inherent values (only maximizes reward)
- Specification of reward function is critical
- Superintelligence would be very capable but not necessarily aligned
- Need robust value alignment before intelligence explosion

---

# Summary and Key Takeaways

## Core Concepts Implemented

1. **Psychometric Intelligence (g-factor)**
   - Simulated cognitive test data
   - Extracted general intelligence factor using PCA
   - Showed positive manifold in correlation matrix

2. **Solomonoff Induction**
   - Approximated universal prior using program enumeration
   - Demonstrated sequence prediction weighted by simplicity
   - Estimated Kolmogorov complexity of sequences

3. **AIXI Agent**
   - Implemented MC-AIXI using MCTS
   - Tested in toy grid-world environments
   - Compared to random and greedy baselines

4. **Universal Intelligence Measure (Υ)**
   - Created environment suite with varying complexities
   - Computed Υ(π) for different agents
   - Showed intelligence emerges from diverse performance

5. **Computational Limits**
   - Explored computation-performance tradeoffs
   - Demonstrated diminishing returns with increased budget
   - Illustrated incomputability of perfect AIXI

6. **Recursive Self-Improvement**
   - Simulated agent improving its own capabilities
   - Modeled different intelligence explosion scenarios
   - Discussed implications for superintelligence

## Philosophical Implications

- **Intelligence is formally definable** (not just intuitive)
- **Simplicity matters** (Solomonoff prior = Occam's razor)
- **Optimality is incomputable** (fundamental limitation)
- **Self-improvement creates feedback loops** (intelligence explosion possible)

## Practical Lessons

- Real AI systems use **approximations** of theoretical ideals
- **Computation budget** critically affects performance
- **Environment diversity** tests true intelligence
- **Value alignment** needed before superintelligence

---

**Further Reading**:
- Legg, S. (2008). *Machine Super Intelligence*. PhD Thesis.
- Hutter, M. (2005). *Universal Artificial Intelligence*.
- Solomonoff, R. (1964). *A Formal Theory of Inductive Inference*.
- Bostrom, N. (2014). *Superintelligence: Paths, Dangers, Strategies*.

**Related Papers**:
- Paper 23: MDL (Minimum Description Length)
- Paper 25: Kolmogorov Complexity  
- Paper 8: DQN (Practical deep RL)